In [4]:
for index,i in enumerate(['sd','sds']):
    print(inf,i)

0 sd
1 sds


In [5]:
from scipy.spatial.distance import jensenshannon

In [7]:
import numpy as np

In [11]:
np.random.randn(2, 4)

array([[ 0.3884779 , -0.10282707, -0.28046676,  0.94322203],
       [ 1.47525405, -0.15213832,  1.22995207, -0.27261961]])

In [43]:
gaussian_filter1d(np.concatenate((np.random.randn(2, 4),np.array([0.0,0,0,0]).reshape(1,4))),sigma=0.5,axis=1)

array([[ 0.75960347,  0.02251224,  1.59823904,  0.13233961],
       [ 0.80117815, -0.62995394,  0.816476  , -0.20195428],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [64]:
pred_profile = np.random.randn(50, 1000,2).reshape(-1,2000)

In [59]:
pred_profile.shape[0]

2000

In [60]:
pred_profile_prob = pred_profile / np.sum(pred_profile,axis=1).reshape(pred_profile.shape[0],1)

In [62]:
pred_profile_prob.shape

(50, 2000)

In [53]:
pred_profile/(np.sum(pred_profile,axis=1).reshape(3,1))

/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[-5.68641609, -1.5522246 ,  5.04368528,  3.19495541],
       [ 0.66093011,  0.46116659, -0.2454665 ,  0.1233698 ],
       [        nan,         nan,         nan,         nan]])

In [50]:
np.sum(pred_profile,axis=1)

array([-0.11202276, -1.56859551,  0.        ])

In [38]:
from scipy.ndimage import gaussian_filter1d
??gaussian_filter1d

Signature:
gaussian_filter1d(
    input,
    sigma,
    axis=-1,
    order=0,
    output=None,
    mode='reflect',
    cval=0.0,
    truncate=4.0,
)
Docstring:
1-D Gaussian filter.

Parameters
----------
input : array_like
    The input array.
sigma : scalar
    standard deviation for Gaussian kernel
axis : int, optional
    The axis of `input` along which to calculate. Default is -1.
order : int, optional
    An order of 0 corresponds to convolution with a Gaussian
    kernel. A positive order corresponds to convolution with
    that derivative of a Gaussian.
output : array or dtype, optional
    The array in which to place the output, or the dtype of the
    returned array. By default an array of the same dtype as input
    will be created.
mode : {'reflect', 'constant', 'nearest', 'mirror', 'wrap'}, optional
    The `mode` parameter determines how the input array is extended
    beyond its boundaries. Default is 'reflect'. Behavior for each valid
    value is as follows:

    'refle